In [196]:
# Script to Convert ROS => Timeseries Format

In [197]:
import pandas as pd
import os
import regex as re
import datetime
import time
import numpy as np

In [198]:
participant = 18
data_path = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/p'+str(participant) +'/Interm2'
out = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/ROS/p'+str(participant) +'/Final/'

In [199]:
def to_seconds(s):
    if s=='0':
        return 0
    
    s = s[:-4]
    
    # First Check if Bigger than 1 Hour
    if len(s)==5:
        test = int(s[0:2])
        if test>=60:
            hours = str(int(test/60))
            minutes = str(test%60)

            if len(minutes) == 1:
                minutes = '0' + minutes
            s = hours + ':' + minutes + s[2:]

    if len(s) > 5:
        regex = '%H:%M:%S'
    else:
        regex = '%M:%S'

    x = time.strptime(s, regex)
    return datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()

In [200]:
for subdir, dirs, files in os.walk(data_path):
    for file in files:
        if 'ros' in file and 'csv' in file:
            print(file)
            sess = pd.read_csv(data_path+'/'+file, header=None)
            sess = sess.rename({0:'start_time', 1:'end_time', 2:'tier', 3:'value'}, axis='columns')
            
            # convert start & end times to seconds
            sess['start_sec'] = np.nan
            sess['end_sec'] = np.nan

            for i,r in sess.iterrows():
                sess.loc[i, 'start_sec'] = to_seconds(r['start_time'])
                sess.loc[i, 'end_sec'] = to_seconds(r['end_time'])
                
            # create answer dataframe
            answer = pd.DataFrame(columns=['timestamp','ROBOT STATE', 'PARTICIPANT STATE', 'GAME STATE'])

            times = [] 
            curr = 0
            interval = 1/30
            last = max(sess['end_sec'])
            while curr <= last+1:
                times.append(curr) 
                curr += interval
            answer['timestamp'] = times
            answer['timestamp'] = round(answer['timestamp'], 5)
            
            # map each tier to timeseries
            for tier in sess['tier'].unique():
                subset = sess[sess['tier']==tier]
                for i,r in subset.iterrows():
                    start = r['start_sec']
                    end = r['end_sec']
                    val = r['value']

                    s_index = answer[answer['timestamp'] == start].index[0]
                    e_index = answer[answer['timestamp'] == end].index[0]

                    answer.loc[s_index:e_index-1, tier] = val
            
            # Save File
            
            # session for file name
            sess_regex1 = 's\d*_'
            sess_regex2 = 's\d*-\d*_'

            res1 = re.findall(sess_regex1, file)
            res2 = re.findall(sess_regex2, file)

            if (res2):
                session = res2[0]
            else:
                session = res1[0]
            
            answer.to_csv(out+'p'+str(participant)+'_'+session+'ros.csv', index=False)

p18_s18_new_ros_2019-01-09-19-00-59_2019-01-09-19-00-59.csv
p18_s17_new_ros_2019-01-05-07-29-25_2019-01-05-07-29-26.csv
p18_s19_new_ros_2019-01-12-14-47-15_2019-01-12-14-47-16.csv
p18_s9_new_ros_2018-12-16-18-56-51_2018-12-16-18-56-52.csv
p18_s10_new_ros_2018-12-18-19-51-00_2018-12-18-19-51-00.csv
p18_s7_new_ros_2018-12-11-19-29-19_2018-12-11-19-29-19.csv
p18_s12_new_ros_2018-12-27-18-58-05_2018-12-27-18-58-06.csv
p18_s15_new_ros_2019-01-02-20-16-17_2019-01-02-20-16-17.csv
p18_s11_new_ros_2018-12-24-15-43-43_2018-12-24-15-43-44.csv
p18_s8_new_ros_2018-12-13-19-58-02_2018-12-13-19-58-03.csv
p18_s5_new_ros_2018-12-09-19-36-03_2018-12-09-19-36-03.csv
p18_s4_new_ros_2018-12-07-20-28-37_2018-12-07-20-28-38.csv
p18_s3_new_ros_2018-12-06-19-27-56_2018-12-06-19-27-56.csv
